In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [2]:
data = pd.read_csv("/kaggle/input/stocknews/Combined_News_DJIA.csv")

In [3]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [4]:
len(test),len(train)

(378, 1611)

In [5]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [6]:
trainvect = CountVectorizer()

In [7]:
Trainfeature = trainvect.fit_transform(trainheadlines)

In [9]:
####Detailed view of Document Count Matrix
DTM_With_Colm = pd.DataFrame(Trainfeature.toarray(),columns= trainvect.get_feature_names_out())

In [10]:
Trainfeature.shape

(1611, 31675)

In [11]:
Logis = LogisticRegression()

In [12]:
Model1 = Logis.fit(Trainfeature,train['Label'])

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
Model1

LogisticRegression()

In [15]:
testheadlines =[]
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [16]:
len(testheadlines)

378

In [17]:
Testfeature = trainvect.transform(testheadlines)

In [18]:
Testfeature.shape

(378, 31675)

In [19]:
Predicted = Model1.predict(Testfeature)

In [20]:
pd.crosstab(test["Label"], Predicted , rownames=["Actual"] , colnames= ["Predict"])

Predict,0,1
Actual,,
0,61,125
1,92,100


In [21]:
from sklearn.naive_bayes import MultinomialNB

Nb = MultinomialNB()

In [22]:
Model2 = Nb.fit(Trainfeature,train['Label'])

In [23]:
Nbpredicted = Model2.predict(Testfeature)

In [24]:
Nbpredicted.shape

(378,)

In [25]:
pd.crosstab(test["Label"], Nbpredicted, rownames= ["Acutal"],colnames=["Predict"])

Predict,0,1
Acutal,,
0,46,140
1,51,141


In [26]:
import numpy as np
from sklearn.metrics import accuracy_score
y_NaviBayes = Nbpredicted
y_true = test["Label"]
accuracy_score(y_NaviBayes,y_true)
x_Logist = Predicted
x_true = test["Label"]
accuracy_score(x_Logist,x_true)

0.42592592592592593

In [29]:
advvect = CountVectorizer(ngram_range=(1,2))
%time
advancedtrain = advvect.fit_transform(trainheadlines)
advancedtrain.shape

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


(1611, 398396)

In [28]:
advmodel = MultinomialNB()
advancemodel = advmodel.fit(advancedtrain,train["Label"])
advancetest = advvect.transform(testheadlines)
advNBprediction = advmodel.predict(advancetest) 
advNBprediction.shape

(378,)

In [30]:
pd.crosstab(test["Label"],advNBprediction, rownames=["Acutal"],colnames=["Predicted"])

Predicted,0,1
Acutal,,
0,3,183
1,0,192


In [31]:
x_adNB = advNBprediction
x_test = test["Label"]

In [32]:
accuracy_score(x_test,x_adNB)

0.5158730158730159

In [33]:
from sklearn import metrics
metrics.accuracy_score(x_test,x_adNB)

0.5158730158730159

In [34]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [35]:
%time
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
#Our Document
trainheadlines

# list for tokenized documents in loop
texts = []

# loop through document list
for i in trainheadlines:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


In [36]:
%time
#generate LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=1,chunksize=10000,update_every=1)


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


In [37]:
ldamodel.print_topics(5)

[(9,
  '0.014*"b" + 0.014*"s" + 0.006*"year" + 0.006*"say" + 0.005*"protest" + 0.004*"us" + 0.004*"will" + 0.004*"kill" + 0.004*"israel" + 0.003*"polic"'),
 (3,
  '0.015*"s" + 0.007*"b" + 0.005*"say" + 0.005*"kill" + 0.004*"polic" + 0.004*"world" + 0.004*"year" + 0.003*"new" + 0.003*"will" + 0.003*"protest"'),
 (6,
  '0.055*"b" + 0.016*"s" + 0.006*"say" + 0.006*"us" + 0.005*"israel" + 0.005*"year" + 0.004*"kill" + 0.004*"will" + 0.004*"world" + 0.003*"china"'),
 (2,
  '0.015*"s" + 0.013*"b" + 0.005*"say" + 0.005*"govern" + 0.005*"new" + 0.004*"kill" + 0.004*"world" + 0.004*"year" + 0.004*"will" + 0.004*"us"'),
 (1,
  '0.018*"s" + 0.007*"b" + 0.006*"say" + 0.006*"us" + 0.004*"year" + 0.004*"govern" + 0.004*"israel" + 0.004*"new" + 0.004*"kill" + 0.003*"peopl"')]

In [38]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
news = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary)

/opt/conda/lib/python3.10/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [39]:
news

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.022049  0.000043       1        1  35.274870
0     -0.004658  0.002575       2        1  22.067492
9     -0.001612  0.002374       3        1  11.199731
5      0.003749  0.003191       4        1  10.857483
4      0.003354 -0.000748       5        1   4.617599
2      0.000479  0.001232       6        1   4.581403
8      0.009461  0.000185       7        1   4.220917
3      0.006840 -0.006458       8        1   2.871990
1      0.007786  0.003693       9        1   2.541674
7     -0.003350 -0.006088      10        1   1.766841, topic_info=        Term          Freq         Total Category  logprob  loglift
136        s   8404.000000   8404.000000  Default  30.0000  30.0000
18         b  14253.000000  14253.000000  Default  29.0000  29.0000
139      say   2621.000000   2621.000000  Default  28.0000  28.0000
89      kill   2124.000000   2124.000000  Default  27.0000  27.0000
674   govern   1770.000000   1770.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
40   countri     22.855519   1500.198414  Topic10  -5.9764  -0.1482
117    peopl     22.271310   1533.397709  Topic10  -6.0023  -0.1960
134   russia     21.203671   1312.235577  Topic10  -6.0514  -0.0893
79      iran     19.175695   1185.803577  Topic10  -6.1519  -0.0886
0        000     19.124059   1215.093786  Topic10  -6.1546  -0.1157

[869 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
0          1  0.358820      000
0          2  0.251009      000
0          3  0.081475      000
0          4  0.115217      000
0          5  0.050202      000
...      ...       ...      ...
18132      4  0.570534     zeid
1755       1  0.670655  zionism
1755       2  0.134131  zionism
1755       3  0.067065  zionism
1755       4  0.067065  zionism

[2320 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 10, 6, 5, 3, 9, 4, 2, 8])